## Imports

In [7]:
import pandas as pd
import os

# Step 1: Dataset path from download
dataset_path = "/Users/husnainsyed/.cache/kagglehub/datasets/arevel/chess-games/versions/1"

# Find the CSV file
csv_file = next(f for f in os.listdir(dataset_path) if f.endswith(".csv"))
csv_path = os.path.join(dataset_path, csv_file)

# Load a memory-friendly subset (all columns)
df = pd.read_csv(
    csv_path,
    nrows=300_000  # still limit rows to avoid memory issues
)




In [8]:
df_event = df['Event']=='Classical'